In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
df=pd.read_csv('/Users/chandan/Desktop/MyLA Data Project/MyLA_event_log_Fall2019-ANONYMIZED.csv')
df.head()

,id,timestamp,action,extra,username,course_id
0,8699,11/15/19 5:23,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001244,course_000000
1,8892,11/18/19 15:51,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001244,course_000000
2,8682,11/15/19 5:09,VIEW_GRADE_DISTRIBUTION,"{""course_id"": ""course_000000"", ""show_number_on...",subject_001228,course_000000
3,6716,10/16/19 14:01,VIEW_RESOURCE_ACCESS,"{""week_num_start"": 4, ""week_num_end"": 6, ""grad...",subject_000911,course_000000
4,6717,10/16/19 14:02,VIEW_ASSIGNMENT_PLANNING,"{""course_id"": ""course_000000"", ""percent_select...",subject_000911,course_000000


In [3]:
stud = pd.read_csv('/Users/chandan/Desktop/MyLA Data Project/MyLA_Student_Data_Fall2019_withCanvasCourses-ANONYMIZED copy 2.csv')
stud.head()


,Campus ID,Sex,International or Domestic,Acad Level BOT,Cum GPA,Math Plcmt,CanvasCourseID_long
0,subject_001637,Male,International,Grad Mastr,3.673,NaN,course_000003
1,subject_001146,Female,Domestic,Grad Mastr,3.731,NaN,course_000003
2,subject_001546,Male,Domestic,Grad Mastr,3.877,25.0,course_000003
3,subject_001376,Female,Domestic,Grad Mastr,3.879,17.0,course_000003
4,subject_000932,Male,Domestic,Grad Mastr,3.770,12.0,course_000003


In [4]:
TOT_STUD_LS = list(set(stud['Campus ID'].to_list())) 


In [5]:
users_df = df.loc[df['username'].isin(TOT_STUD_LS) ]
users_df.username.nunique()


421

In [6]:
### ALLOCATING THE VIEW_SET_DEFAULT'S CORRESPONDING ACTION FROM EXTRA COLUMN TO THE ACTION 
vsd_dict = {'grade':'VIEW_GRADE_DISTRIBUTION', 'resource' : 'VIEW_RESOURCE_ACCESS','assignment': 'VIEW_ASSIGNMENT_PLANNING'}

tmp2 = users_df.loc[users_df['action'] == 'VIEW_SET_DEFAULT']
tmp2['vsd_cat'] = tmp2['extra'].apply(lambda x: x.split('default_type":')[1].split(',')[0].strip().replace('\"','') )

users_df.loc[users_df['action'] == 'VIEW_SET_DEFAULT','action'] =  tmp2['vsd_cat'].map(vsd_dict).to_list() 


/var/folders/nn/xh7kbss534gc8ynjxk4mz5hr0000gn/T/ipykernel_1182/3829436437.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp2['vsd_cat'] = tmp2['extra'].apply(lambda x: x.split('default_type":')[1].split(',')[0].strip().replace('\"','') )
/Users/chandan/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [7]:
## students in log sheet that are not in student database -- NEED TO BE CLARIFIED
non_users_df1 = df.loc[~ df['username'].isin(TOT_STUD_LS) ]
non_users_df1.username.nunique()


10

In [8]:
# Usernames in the log file that are not present in the student database
NON_USERS_LS1 =non_users_df1.username.unique()
NON_USERS_LS2 = [user for user in TOT_STUD_LS if user not in users_df.username.unique()]

NON_USERS_LS = list(set(list(NON_USERS_LS1) + list( NON_USERS_LS2) ))


In [9]:
tmp = users_df.groupby(['username','action']).count() ['id'].reset_index()
module_user_cnt_df = tmp.pivot(index='username',columns='action',values = 'id').fillna(0).reset_index()
print(module_user_cnt_df.shape)
module_user_cnt_df.head()

(421, 4)


action,username,VIEW_ASSIGNMENT_PLANNING,VIEW_GRADE_DISTRIBUTION,VIEW_RESOURCE_ACCESS
0,subject_000115,0.0,3.0,0.0
1,subject_000179,2.0,13.0,1.0
2,subject_000339,0.0,1.0,0.0
3,subject_000363,0.0,2.0,1.0
4,subject_000398,4.0,14.0,2.0


In [10]:
module_user_cnt_df.rename(columns = {'VIEW_ASSIGNMENT_PLANNING':'VAP',
                                    'VIEW_GRADE_DISTRIBUTION' : 'VGD',
                                    'VIEW_RESOURCE_ACCESS' : 'VRA',
                                    'VIEW_SET_DEFAULT' : 'VSD'}, inplace= True)

In [11]:
# 20 10 8 4 <2
def get_category(df,COL_NAMES_LS):
    OP_LS = []
    my_df = df.copy()
    for COL_NAME in COL_NAMES_LS:
        TMP = pd.cut(x=my_df[COL_NAME], bins=[-1, 10, 20, 30, 40,100],
                    labels=['L' ,'N', 'M','H','SH']).fillna('SH').reset_index()
        op = TMP[COL_NAME].to_list()
        OP_LS.append(op)
#     print(OP_LS)
    TMP = pd.DataFrame(OP_LS ).T
    TMP.columns = [col+'_CATEGORY' for col in COL_NAMES_LS]
    RET_DF = pd.concat([my_df,TMP],axis=1)
    return RET_DF

In [12]:
MODULE_LS = ['VAP', 'VGD','VRA'] #,'VSD'

In [13]:
categorized_users = get_category(module_user_cnt_df,MODULE_LS)
categorized_users

,username,VAP,VGD,VRA,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY
0,subject_000115,0.0,3.0,0.0,L,L,L
1,subject_000179,2.0,13.0,1.0,L,N,L
2,subject_000339,0.0,1.0,0.0,L,L,L
3,subject_000363,0.0,2.0,1.0,L,L,L
4,subject_000398,4.0,14.0,2.0,L,N,L
...,...,...,...,...,...,...,...
416,subject_001750,0.0,4.0,0.0,L,L,L
417,subject_001754,0.0,3.0,0.0,L,L,L
418,subject_001756,0.0,1.0,0.0,L,L,L
419,subject_001757,0.0,8.0,0.0,L,L,L


In [14]:
categorized_users.VAP_CATEGORY.value_counts()

L    416
N      4
M      1
Name: VAP_CATEGORY, dtype: int64

In [15]:
code_dict = {'SH':5,'H':4,'M': 3,'N':2,'L': 1}

In [16]:
for i in MODULE_LS:
    categorized_users[i+'_SCORE'] = categorized_users[i+'_CATEGORY'].map(code_dict)

In [17]:
categorized_users['TOTAL_SCORE'] = categorized_users.apply(lambda x: sum([x[i+'_SCORE'] for i in MODULE_LS]), axis=1)
categorized_users

,username,VAP,VGD,VRA,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,TOTAL_SCORE
0,subject_000115,0.0,3.0,0.0,L,L,L,1,1,1,3
1,subject_000179,2.0,13.0,1.0,L,N,L,1,2,1,4
2,subject_000339,0.0,1.0,0.0,L,L,L,1,1,1,3
3,subject_000363,0.0,2.0,1.0,L,L,L,1,1,1,3
4,subject_000398,4.0,14.0,2.0,L,N,L,1,2,1,4
...,...,...,...,...,...,...,...,...,...,...,...
416,subject_001750,0.0,4.0,0.0,L,L,L,1,1,1,3
417,subject_001754,0.0,3.0,0.0,L,L,L,1,1,1,3
418,subject_001756,0.0,1.0,0.0,L,L,L,1,1,1,3
419,subject_001757,0.0,8.0,0.0,L,L,L,1,1,1,3


In [18]:
# categorized_users['VGD_SCORE']= categorized_users['VGD_SCORE']+categorized_users['VSD_SCORE']
# categorized_users

In [19]:
# del categorized_users['VSD_SCORE']
# categorized_users

In [20]:
Dict_overall_perf = {'SUPER HIGH USERS':5,'SUPER USERS':4,'HIGH USERS': 3,'MEDIUM USERS':2,'LOW USERS': 1, 'NON USERS': 0}

In [21]:
categorized_users['OVERALL_PROFILE'] = pd.cut(x= categorized_users['TOTAL_SCORE'], bins=[-1, 2, 4, 7, 15,100],labels=['LOW USERS' ,'NORMAL USERS', 'MEDIUM USERS','HIGH USERS','SUPER HIGH USERS'])

In [22]:
categorized_users['OVERALL_PROFILE'].value_counts()

NORMAL USERS        392
MEDIUM USERS         25
HIGH USERS            4
LOW USERS             0
SUPER HIGH USERS      0
Name: OVERALL_PROFILE, dtype: int64

In [23]:
categorized_users.head()

,username,VAP,VGD,VRA,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,TOTAL_SCORE,OVERALL_PROFILE
0,subject_000115,0.0,3.0,0.0,L,L,L,1,1,1,3,NORMAL USERS
1,subject_000179,2.0,13.0,1.0,L,N,L,1,2,1,4,NORMAL USERS
2,subject_000339,0.0,1.0,0.0,L,L,L,1,1,1,3,NORMAL USERS
3,subject_000363,0.0,2.0,1.0,L,L,L,1,1,1,3,NORMAL USERS
4,subject_000398,4.0,14.0,2.0,L,N,L,1,2,1,4,NORMAL USERS


In [24]:
stud.rename(columns={'Campus ID': 'username', 'Sex': 'Gender','International or Domestic': 'Immigration',
                     'Cum GPA': 'CGPA'}, inplace= True)
## we just need student wise data so drop duplicates based on username in stud 
my_stud = stud.loc[:,['username','Immigration','Gender','CGPA']].drop_duplicates('username')

In [25]:
hyp1_df = categorized_users.merge(my_stud, how='left', on='username')
hyp1_df.head()

,username,VAP,VGD,VRA,VAP_CATEGORY,VGD_CATEGORY,VRA_CATEGORY,VAP_SCORE,VGD_SCORE,VRA_SCORE,TOTAL_SCORE,OVERALL_PROFILE,Immigration,Gender,CGPA
0,subject_000115,0.0,3.0,0.0,L,L,L,1,1,1,3,NORMAL USERS,Domestic,Male,3.523
1,subject_000179,2.0,13.0,1.0,L,N,L,1,2,1,4,NORMAL USERS,Domestic,Male,3.711
2,subject_000339,0.0,1.0,0.0,L,L,L,1,1,1,3,NORMAL USERS,Domestic,Female,3.413
3,subject_000363,0.0,2.0,1.0,L,L,L,1,1,1,3,NORMAL USERS,Domestic,Female,3.751
4,subject_000398,4.0,14.0,2.0,L,N,L,1,2,1,4,NORMAL USERS,Domestic,Male,3.512


In [26]:
hyp1_res = hyp1_df.groupby(['OVERALL_PROFILE','Gender','Immigration']).mean()['CGPA'].fillna(0).reset_index()
tmp1 = hyp1_res.pivot(index='OVERALL_PROFILE', columns=['Gender','Immigration'])['CGPA']

check = my_stud.loc[my_stud['username'].isin(NON_USERS_LS)]
hyp1_res_2 = check.groupby(['Gender','Immigration']).mean()['CGPA'].fillna(0).reset_index()
hyp1_res_2['OVERALL_PROFILE'] = 'NON_USERS'
tmp2 = hyp1_res_2.pivot(index='OVERALL_PROFILE', columns=['Gender','Immigration'])['CGPA']

hyp1 = pd.concat([tmp1,tmp2])
hyp1

Gender              Female                    Male              
Immigration       Domestic International  Domestic International
OVERALL_PROFILE                                                 
LOW USERS         0.000000      0.000000  0.000000      0.000000
NORMAL USERS      3.649460      3.729281  3.442402      3.673641
MEDIUM USERS      3.457917      3.486000  3.493500      3.733600
HIGH USERS        0.000000      4.128000  3.417000      3.742000
SUPER HIGH USERS  0.000000      0.000000  0.000000      0.000000
NON_USERS         3.584750      3.722860  3.357010      3.541986

## HYPOTHESIS 2 - COURSE LEVEL ANALYSIS

In [27]:
## users
check1 = users_df.loc[~users_df['username'].isin(NON_USERS_LS1)].drop_duplicates(['username','course_id'])
check1 = check1.loc[:, ['username','course_id']].copy()
print(check1.shape)
check = check1.merge(my_stud, how='left', on='username')
print('check:', check.shape)

## users course wise count
tmp1 = check.groupby('course_id').count()['username']

## users avg cgpa
hyp2_res_2 = check.groupby(['course_id','Gender','Immigration']).mean()['CGPA'].fillna(0).reset_index()
tmp2 = hyp2_res_2.pivot(index='course_id', columns=['Gender','Immigration'])['CGPA'].fillna(0)

u_summary = pd.concat([tmp1,tmp2],axis=1)
u_summary

(435, 2)
check: (435, 5)


,username,"(Female, Domestic)","(Female, International)","(Male, Domestic)","(Male, International)"
course_id,,,,,
course_000000,130,3.411390,3.454167,3.282097,3.481727
course_000001,3,4.055333,0.000000,0.000000,0.000000
course_000002,5,3.738000,0.000000,3.725333,0.000000
course_000003,34,3.886400,3.894250,3.829556,4.000000
course_000004,43,3.415292,0.000000,3.359368,0.000000
course_000005,3,0.000000,3.925000,0.000000,3.850000
course_000006,1,3.746000,0.000000,0.000000,0.000000
course_000007,56,3.743667,3.788000,3.736286,3.723083
course_000008,12,3.927000,0.000000,4.094000,0.000000


In [28]:
u_summary.username.sum()

435

In [29]:
## non-users - course wise summary - 
## considering stud: student,course id wise summary   
## NOT my_stud: student wise summary
check = stud.loc[stud['username'].isin(NON_USERS_LS)]

## non users course wise count
tmp1 = check.groupby('CanvasCourseID_long').count()['username']

## non-users avg cgpa
hyp2_res_2 = check.groupby(['CanvasCourseID_long','Gender','Immigration']).mean()['CGPA'].fillna(0).reset_index()
tmp2 = hyp2_res_2.pivot(index='CanvasCourseID_long', columns=['Gender','Immigration'])['CGPA'].fillna(0)

nu_summary = pd.concat([tmp1,tmp2],axis=1)
nu_summary

,username,"(Female, Domestic)","(Female, International)","(Male, Domestic)","(Male, International)"
CanvasCourseID_long,,,,,
course_000000,107,3.102486,3.446000,3.023050,3.344000
course_000001,8,3.922375,0.000000,0.000000,0.000000
course_000002,11,3.988333,0.000000,3.772000,0.000000
course_000003,28,3.772632,4.058000,3.704429,3.673000
course_000004,25,3.378235,0.000000,3.539875,0.000000
course_000005,10,3.972000,3.855250,0.000000,0.642200
course_000006,103,3.510140,3.900000,3.226745,3.795250
course_000007,88,3.717750,3.677636,3.806852,3.668727
course_000008,24,3.787048,4.171000,3.526500,0.000000


In [30]:
nu_summary.username.sum()

630

In [31]:
Overall_Course_Summary = pd.concat([u_summary,nu_summary],axis=1)
Overall_Course_Summary

,username,"(Female, Domestic)","(Female, International)","(Male, Domestic)","(Male, International)",username,"(Female, Domestic)","(Female, International)","(Male, Domestic)","(Male, International)"
course_000000,130.0,3.411390,3.454167,3.282097,3.481727,107,3.102486,3.446000,3.023050,3.344000
course_000001,3.0,4.055333,0.000000,0.000000,0.000000,8,3.922375,0.000000,0.000000,0.000000
course_000002,5.0,3.738000,0.000000,3.725333,0.000000,11,3.988333,0.000000,3.772000,0.000000
course_000003,34.0,3.886400,3.894250,3.829556,4.000000,28,3.772632,4.058000,3.704429,3.673000
course_000004,43.0,3.415292,0.000000,3.359368,0.000000,25,3.378235,0.000000,3.539875,0.000000
course_000005,3.0,0.000000,3.925000,0.000000,3.850000,10,3.972000,3.855250,0.000000,0.642200
course_000006,1.0,3.746000,0.000000,0.000000,0.000000,103,3.510140,3.900000,3.226745,3.795250
course_000007,56.0,3.743667,3.788000,3.736286,3.723083,88,3.717750,3.677636,3.806852,3.668727
course_000008,12.0,3.927000,0.000000,4.094000,0.000000,24,3.787048,4.171000,3.526500,0.000000
course_000009,14.0,3.698333,0.000000,3.385500,0.000000,37,3.726963,0.000000,3.297200,0.000000
